In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL del ecommerce
url = 'https://www.amazon.es/gp/bestsellers/?ref_=nav_cs_bestsellers'
# url = 'https://www.amazon.es/b/?_encoding=UTF8&node=76234352031&pd_rd_w=1YVyv&content-id=amzn1.sym.5ad5b890-ecb9-4ef5-8484-3678523450dc&pf_rd_p=5ad5b890-ecb9-4ef5-8484-3678523450dc&pf_rd_r=8NR2MVGA1N49VS8YKG6K&pd_rd_wg=q8U6B&pd_rd_r=dd713508-3446-42f4-8643-a72f56810be7&ref_=pd_hp_d_atf_unk&discounts-widget=%2522%257B%255C%2522state%255C%2522%253A%257B%255C%2522refinementFilters%255C%2522%253A%257B%257D%257D%252C%255C%2522version%255C%2522%253A1%257D%2522'
# Obtener el contenido HTML de la página
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Encontrar todos los productos
clase_main_bestsellers = "a-link-normal aok-block"
clase_historial_bestsellers = "a-link-normal"
productos_main = soup.find_all('a', class_=clase_main_bestsellers)
productos_historial = soup.find_all('a', class_=clase_historial_bestsellers)
productos = productos_historial + productos_main

In [ ]:
from urllib.parse import urljoin

# prueba = prueba.get('href')
base_url = 'https://www.amazon.com'
urls = []
for producto in productos:
    enlace_absoluto = urljoin(base_url, producto.get('href'))
    urls.append(enlace_absoluto)
len(urls)

In [ ]:
# Obtener el contenido HTML de la página del producto
clase = "a-expander-content reviewText review-text-content a-expander-partial-collapse-content"

todas_las_reseñas = []
for url in urls:
    response_producto = requests.get(url)
    soup_producto = BeautifulSoup(response_producto.text, 'html.parser')
    reseñas = soup_producto.find_all('div', class_=clase)
    
    reseñas_limpias = [] # Para obtener solo el contenido en texto dentro del span que queremos
    for reseña in reseñas:
        contenido_span = reseña.find('span').text
        reseñas_limpias.append(contenido_span)

    todas_las_reseñas.append(reseñas_limpias) # añadimos dentro de la lista una lista con todas las reseñas de cada producto


In [ ]:
lista_final = []
for i, k in enumerate(todas_las_reseñas):
    for j, z in enumerate(k):
        if len(z) != 0:
            lista_final.append(z) 

In [ ]:
dt = pd.DataFrame(lista_final)

In [ ]:
dt

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup
import pandas as pd

In [159]:
# Sacamos todas las rutas de "ver mas" para obtener la mayor cantidad de productos posibles

# Ruta al chromedriver
chromedriver_path = r'C:\Users\pabma\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'

# Inicializa el navegador
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

driver.get('https://www.amazon.es/gp/bestsellers/?ref_=nav_cs_bestsellers')
driver.maximize_window()

# Acceptamos las cookies
accept_cokies = driver.find_element(By.ID ,'sp-cc-accept')
accept_cokies.click()
driver.implicitly_wait(10) # Una espera para evitar errores

# a-link-normal
urls_econtradas = driver.find_elements(By.CLASS_NAME, 'a-link-normal')
urls_ver_mas = []
for url in urls_econtradas:
    if url.text == 'Ver más':
        urls_ver_mas.append(url)


In [ ]:
urls_ver_mas.pop(0)

In [ ]:
urls_ver_mas

In [ ]:
# Probamos a intentar conseguir reseñas de mas de un producto de forma automatizada
def obtener_reseñas(urls_todos_productos, categoria):
    lista_df = []
    for url_producto in urls_todos_productos:
        driver.get(url_producto.get_attribute('href'))
        driver.implicitly_wait(10)

        try:
            clase_producto = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            reseñas_en_div = soup.find_all('div', class_=clase_producto)
            nombre_producto = soup.find('span', id='productTitle').text

            reseñas = []
            for reseña in reseñas_en_div:
                if (reseña.find('span').text != 'Video Player is loading.') and (type(reseña.find('span').text) == str): # Algunas reseñas con video daban este mensaje en lugar de la reseña que de nada nos sirve
                    # y otras nos daban nones, por lo que podemos filtrar un poco
                    reseñas.append(reseña.find('span').text)

            producto = pd.DataFrame()
            producto['reseñas_humanas'] = reseñas
            producto['nombre_producto'] = nombre_producto
            producto['categoria_producto'] = categoria # Esto de momento no cambia
            lista_df.append(producto)
        except Exception as e:
            print(f"Error al extraer información de {url_producto.get_attribute('href')}: {e}") # obsevamos en que url no ha podido entrar

        
        driver.back()
        driver.implicitly_wait(10)

    combined_df = pd.concat(lista_df, ignore_index=True)
    return combined_df

In [ ]:
# BUENO
lista_df = []
clase_link_hijo = 'aok-block'
for url_ver_mas in urls_ver_mas:
    driver.get(url_ver_mas.get_attribute('href'))
    driver.implicitly_wait(10)

    urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
    categoria = categoria_en_div.find('h1').text

    df_ver_mas_individual = obtener_reseñas(urls_todos_productos, categoria) # dentro de este df guardamos todas las reseñas de todos los productos en cada url_ver_mas
    lista_df.append(df_ver_mas_individual)

    driver.back()

driver.quit()

In [ ]:
# BUENO
lista_df_final = []
clase_link_hijo = 'aok-block'
for url_ver_mas in urls_ver_mas:
    driver.get(url_ver_mas.get_attribute('href'))
    driver.implicitly_wait(10)

    urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
    categoria = categoria_en_div.find('h1').text

    lista_df = []
    for url_producto in urls_todos_productos:
        driver.get(url_producto.get_attribute('href'))
        driver.implicitly_wait(10)

        try:
            clase_producto = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            reseñas_en_div = soup.find_all('div', class_=clase_producto)
            nombre_producto = soup.find('span', id='productTitle').text

            reseñas = []
            for reseña in reseñas_en_div:
                if (reseña.find('span').text != 'Video Player is loading.') and (type(reseña.find('span').text) == str): # Algunas reseñas con video daban este mensaje en lugar de la reseña que de nada nos sirve
                    # y otras nos daban nones, por lo que podemos filtrar un poco
                    reseñas.append(reseña.find('span').text)

            producto = pd.DataFrame()
            producto['reseñas_humanas'] = reseñas
            producto['nombre_producto'] = nombre_producto
            producto['categoria_producto'] = categoria # Esto de momento no cambia
            lista_df.append(producto)
        except Exception as e:
            print(f"Error al extraer información de {url_producto.get_attribute('href')}: {e}") # obsevamos en que url no ha podido entrar

        
        driver.back()
        driver.implicitly_wait(10)

    combined_df = pd.concat(lista_df, ignore_index=True)
    lista_df_final.append(combined_df)

    driver.back()

driver.quit()

In [ ]:
# Obtenemos las urls de todos los productos dentro de "ver mas"
clase_link_producto = 'a-link-normal'
clase_link_hijo = 'aok-block'
# clase_link_producto = 'a-link-normal'
urls_ver_mas[0].click()
WebDriverWait(driver, 10)
urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)
# urls_todos_productos = urls_todos_productos_.find_elements(By.CLASS_NAME, clase_link_hijo)
# 3 divs se llaman a-link-normal, uno el de la imagen, otro el del link en si y otro para las reviews del producto, con a-link-normal aok-block solo coge el link de la imagen

# Obtenemos la categoria para mas informacion
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# urls_todos_productos = soup.find_all('a', class_=clase_link_producto)
categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
categoria = categoria_en_div.find('h1').text

In [ ]:
urls_todos_productos[3].get_attribute('href')

In [ ]:
# Obtener todas las reseñas y el nombre del producto
urls_todos_productos[0].click()
clase_producto = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
reseñas_en_div = soup.find_all('div', class_=clase_producto)
nombre_producto = soup.find('span', id='productTitle').text

reseñas = []
for reseña in reseñas_en_div:
    if (reseña.find('span').text != 'Video Player is loading.') and (type(reseña.find('span').text) == str): # Algunas reseñas con video daban este mensaje en lugar de la reseña que de nada nos sirve
        # y otras nos daban nones, por lo que podemos filtrar un poco
        reseñas.append(reseña.find('span').text)

In [ ]:
# Creamos el dataset
producto = pd.DataFrame()
producto['reseñas_humanas'] = reseñas
producto['nombre_producto'] = nombre_producto
producto['categoria_producto'] = categoria
producto

# Ideas: Una opción es tener todas las reseñas en una misma columna y tener otra columna que diga con un 0 o 1 si es ia o no, esta opcion
# podría ser muy interesante para hacer un modelo de clasificación, la otra opción es tener 2 columnas de reseñas, una con la humana y otra con la ia
# creando parejas de reseñas que hablan del mismo producto pero uno escrito por una persona y otro por una ia, este sería interesante para localizar
# diferencias


In [ ]:
# lo que tenemos hasta ahora funciona, sin embargo tenemos que automatizarlo para que recorra todos los links de las categorias y luego
# todos los links de los productos, haga dataframes de cada grupo de reseñas y haga un merge para tener un dataframe mas grande con todas
# las reseñas
driver.quit()

In [ ]:
urls_todos_productos[0].get_attribute('href')

In [ ]:
# Probamos a intentar conseguir reseñas de mas de un producto de forma automatizada
def obtener_reseñas(urls_todos_productos):
    lista_df = []
    for url_producto in urls_todos_productos:
        driver.get(url_producto.get_attribute('href'))
        driver.implicitly_wait(10)

        try:
            clase_producto = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            reseñas_en_div = soup.find_all('div', class_=clase_producto)
            nombre_producto = soup.find('span', id='productTitle').text

            reseñas = []
            for reseña in reseñas_en_div:
                if (reseña.find('span').text != 'Video Player is loading.') and (type(reseña.find('span').text) == str): # Algunas reseñas con video daban este mensaje en lugar de la reseña que de nada nos sirve
                    # y otras nos daban nones, por lo que podemos filtrar un poco
                    reseñas.append(reseña.find('span').text)

            producto = pd.DataFrame()
            producto['reseñas_humanas'] = reseñas
            producto['nombre_producto'] = nombre_producto
            producto['categoria_producto'] = categoria # Esto de momento no cambia
            lista_df.append(producto)
        except Exception as e:
            print(f"Error al extraer información de {url_producto.get_attribute('href')}: {e}") # obsevamos en que url no ha podido entrar

        
        driver.back()
        driver.implicitly_wait(10)

    combined_df = pd.concat(lista_df, ignore_index=True)
    return combined_df

In [160]:
## prueba, con esto obtenemos todas las reseñas de todos los productos dentro de un ver mas, hagamos manualmente el proceso con todas las url_ver_mas
clase_link_producto = 'a-link-normal'
clase_link_hijo = 'aok-block'
urls_ver_mas[0].click()
WebDriverWait(driver, 10)
urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
categoria = categoria_en_div.find('h1').text

lista_df = []
for url_producto in urls_todos_productos:
    driver.get(url_producto.get_attribute('href'))
    driver.implicitly_wait(10)

    try:
        clase_producto = 'a-expander-content reviewText review-text-content a-expander-partial-collapse-content'
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        reseñas_en_div = soup.find_all('div', class_=clase_producto)
        nombre_producto = soup.find('span', id='productTitle').text

        reseñas = []
        for reseña in reseñas_en_div:
            if (reseña.find('span').text != 'Video Player is loading.') and (type(reseña.find('span').text) == str): # Algunas reseñas con video daban este mensaje en lugar de la reseña que de nada nos sirve
                # y otras nos daban nones, por lo que podemos filtrar un poco
                reseñas.append(reseña.find('span').text)

        producto = pd.DataFrame()
        producto['reseñas_humanas'] = reseñas
        producto['nombre_producto'] = nombre_producto
        producto['categoria_producto'] = categoria # Esto de momento no cambia
        lista_df.append(producto)
    except Exception as e:
        print(f"Error al extraer información de {url_producto.get_attribute('href')}: {e}") # obsevamos en que url no ha podido entrar

    
    driver.back()
    driver.implicitly_wait(10)

combined_df = pd.concat(lista_df, ignore_index=True)

Exception ignored in: <function Service.__del__ at 0x0000023621F972E0>
Traceback (most recent call last):
  File "c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 189, in __del__
    self.stop()
  File "c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 146, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 131, in send_remote_shutdown_command
    if not self.is_connectable():
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdriver\common\service.py", line 120, in is_connectable
    return utils.is_connectable(self.port)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\pabma\AppData\Local\Programs\Python\Python312\Lib\site-packages\selenium\webdri

In [ ]:
# BUENO
lista_df = []
clase_link_hijo = 'aok-block'
for url_ver_mas in urls_ver_mas:
    driver.get(url_ver_mas.get_attribute('href'))
    driver.implicitly_wait(10)

    urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
    categoria = categoria_en_div.find('h1').text

    df_ver_mas_individual = obtener_reseñas(urls_todos_productos) # dentro de este df guardamos todas las reseñas de todos los productos en cada url_ver_mas
    lista_df.append(df_ver_mas_individual)

    driver.back()

driver.quit()

In [ ]:
# Ya que podemos sacar las reseñas de todos los productos dentro de un "ver mas" hagamos lo mismo pero dentro de ver mas
lista_df = []
clase_link_hijo = 'aok-block'
for url_ver_mas in urls_ver_mas:
    driver.get(url_ver_mas.get_attribute('href'))
    driver.implicitly_wait(10)

    try:
        urls_todos_productos = driver.find_elements(By.CLASS_NAME, clase_link_hijo)
        driver.implicitly_wait(10)

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        categoria_en_div = soup.find('div', class_='_cDEzb_card-title_2sYgw')
        categoria = categoria_en_div.find('h1').text
        

        df_ver_mas_individual = obtener_reseñas(urls_todos_productos) # dentro de este df guardamos todas las reseñas de todos los productos en cada url_ver_mas
        lista_df.append(df_ver_mas_individual)
        driver.implicitly_wait(10)
    
    except Exception as e:
        print(f"Error al extraer información de {e}") 

    driver.back()
    driver.implicitly_wait(10)


driver.quit()

In [ ]:
len(lista_df)

In [ ]:
urls_ver_mas[0].get_attribute('href')

In [ ]:
url

In [ ]:
urls_todos_productos[2].get_attribute('href')

In [ ]:
lista_df

In [ ]:
combined_df = pd.concat(lista_df, ignore_index=True)

In [161]:
len(combined_df['reseñas_humanas'].unique()) # hay 343 reseñas distintas de 30 productos, es una media de 11.43 reseñas por producto

343

In [162]:
combined_df

,reseñas_humanas,nombre_producto,categoria_producto
0,Funcionan perfectamente luz blanca y bonita pa...,Moxled Guirnaldas Luces Exterior Solar...,Los más vendidos en Iluminación
1,Las compré para el jardin y aguantan todo tipo...,Moxled Guirnaldas Luces Exterior Solar...,Los más vendidos en Iluminación
2,"Son más pequeñas de lo que quería, pero son gr...",Moxled Guirnaldas Luces Exterior Solar...,Los más vendidos en Iluminación
3,Buen producto. Relación calidad- precio muy bu...,Moxled Guirnaldas Luces Exterior Solar...,Los más vendidos en Iluminación
4,Muy chulas,Moxled Guirnaldas Luces Exterior Solar...,Los más vendidos en Iluminación
...,...,...,...
365,We bought this for our 10 year old son for Chr...,"Gritin 19 LED Luz de Lectura, 360° Fle...",Los más vendidos en Iluminación
366,This light is small yet mighty. I love the fe...,"Gritin 19 LED Luz de Lectura, 360° Fle...",Los más vendidos en Iluminación
367,Mi principal problema al leer es encontrar bue...,"Gritin 19 LED Luz de Lectura, 360° Fle...",Los más vendidos en Iluminación
368,Batteriet håller länge efter första användning...,"Gritin 19 LED Luz de Lectura, 360° Fle...",Los más vendidos en Iluminación


In [ ]:
len(urls_ver_mas) # por cada url de ver mas hay unos 343 productos y tenemos 6 urls, por lo tanto tendriamos aproximadamente 2058 muestras

In [ ]:
combined_df['categoria_producto'].unique()

In [ ]:
len(combined_df) - combined_df.duplicated().sum()

In [ ]:
len(combined_df['reseñas_humanas'].unique())

In [ ]:
# Parece que hay un monton de duplicados, por lo visto se repiten 4 veces por algun motivo y parece que solo tiene 30 productos diferentes
# cuando creo que deberia tener 50

In [ ]:
len(urls_todos_productos)

In [ ]:
df_comprobacion = pd.DataFrame()
df_comprobacion['urls'] = urls_todos_productos

In [ ]:
df_comprobacion['urls'].duplicated().sum() # Son todo urls distintas, porque entonces hay tantos pocos uniques en nombre, si tendria que haber las 
# mismas url que nombres de productos aunque haya muchos duplicados

In [ ]:
#########################################################

In [ ]:
for seccion in urls_ver_mas:
    try:
        seccion.click()
        url_productos = driver.find_elements(By.CLASS_NAME, 'a-link-normal')
        for producto in url_productos:
            reseñas_producto = []
            producto.click()
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            reseñas = soup_producto.find_all('div', class_=clase)
    
            reseñas_limpias = [] # Para obtener solo el contenido en texto dentro del span que queremos
            for reseña in reseñas:
                contenido_span = reseña.find('span').text
                reseñas_limpias.append(contenido_span)

            todas_las_reseñas.append(reseñas_limpias)
    except:
        print("Se produjo un error")



In [ ]:
todas_las_reseñas

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Configuración del navegador

# Ruta al chromedriver
chromedriver_path = r'C:\Users\pabma\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'

# Inicializa el navegador
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service)

driver.get('https://www.amazon.es/gp/bestsellers/?ref_=nav_cs_bestsellers')
driver.maximize_window()

# Aceptar cookies si es necesario
try:
    accept_button = driver.find_element(By.ID ,'sp-cc-accept')
    accept_button.click()
except NoSuchElementException:
    pass

# Esperar hasta que los elementos estén presentes
wait = WebDriverWait(driver, 15)

# Encontrar los elementos "Ver más"
urls_ver_mas = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='a-link-normal' and text()='Ver más']")))

for seccion in urls_ver_mas:
    try:
        seccion.click()
        # Esperar a que se carguen los elementos de los productos
        wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[@class='a-link-normal']")))
        url_productos = driver.find_elements(By.XPATH, "//a[@class='a-link-normal']")
        for producto in url_productos:
            reseñas_producto = []
            producto.click()
            # Esperar a que se carguen los elementos de las reseñas
            wait.until(EC.presence_of_all_elements_located((By.XPATH, "//div[@class='a-expander-content reviewText review-text-content a-expander-partial-collapse-content']")))
            reseñas = driver.find_elements(By.XPATH, "//div[@class='a-expander-content reviewText review-text-content a-expander-partial-collapse-content']")
            for reseña in reseñas:
                contenido = reseña.find_element(By.XPATH, ".//span")
                texto = contenido.text
                reseñas_producto.append(texto)
            print(reseñas_producto)  # Opcional: Imprime las reseñas para este producto
    except NoSuchElementException:
        print("No se pudo encontrar el elemento 'Ver más'")
    except TimeoutException:
        print("Tiempo de espera agotado para cargar los elementos")
    finally:
        driver.back()  # Regresa a la página de bestsellers para continuar con la siguiente sección

driver.quit()


In [ ]:
reseñas_producto

In [ ]:
len(urls)

In [ ]:
# dsada
# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# import time

# # Configurar Selenium con el controlador de Chrome
# # service = Service('/Users/pabma/Downloads/chrome-win32/.exe')  # Reemplaza con la ruta a tu ChromeDriver
# # driver = webdriver.Chrome(service=service)

# service = webdriver.ChromeService(port=1234)
# driver = webdriver.Chrome(service=service)
# # driver = webdriver.Chrome()

# # URL de la página
# url = 'https://www.example.com'  # Reemplaza con la URL de tu página

# # Navegar a la página
# driver.get(enlace_absoluto)

# # Esperar a que el contenido se cargue completamente
# # Ajusta el tiempo de espera según sea necesario
# wait = WebDriverWait(driver, 10)
# review_div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-a-expander-name="review_text_read_more"]')))

# # Hacer clic en el botón para expandir la reseña si es necesario
# expand_button = review_div.find_element(By.CSS_SELECTOR, '.a-expander-header')
# expand_button.click()

# # Esperar a que el contenido expandido se cargue
# expanded_content = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-hook="review-collapsed"]')))

# # Encontrar el <span> dentro del div expandido
# span = expanded_content.find_element(By.CSS_SELECTOR, 'span')

# # Obtener el texto del <span>
# span_text = span.text

# # Imprimir el contenido del <span>
# print(span_text)

# # Cerrar el navegador
# driver.quit()


In [ ]:
# from selenium import webdriver


# def test_basic_service():
#     service = webdriver.ChromeService()
#     driver = webdriver.Chrome(service=service)

#     driver.quit()


# def test_driver_location(chromedriver_bin, chrome_bin):
#     options = webdriver.ChromeOptions()
#     options.binary_location = chrome_bin

#     service = webdriver.ChromeService(executable_path=chromedriver_bin)

#     driver = webdriver.Chrome(service=service, options=options)

#     driver.quit()


# def test_driver_port():
#     service = webdriver.ChromeService(port=1234)

#     driver = webdriver.Chrome(service=service)

#     driver.quit()


# test_driver_port()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

service = webdriver.ChromeService(port=1234)
driver = webdriver.Chrome(service=service)

try:
    # Navega a la página de e-commerce
    driver.get('https://www.amazon.es/gp/bestsellers/?ref_=nav_cs_bestsellers')

    # Espera a que la página cargue completamente
    time.sleep(5)

    # Localiza el botón de la flecha (esto puede variar según la estructura del sitio web)
    flecha_derecha = driver.find_elements(By.CLASS_NAME, 'a-button a-button-image a-carousel-button a-carousel-goto-nextpage')

    # Haz clic en la flecha para desplazar el carrusel
    NUMERO_DE_CLICKS = 1
    for i in flecha_derecha:
        i.click()
        time.sleep(2)  # Espera un poco para que el carrusel se desplace

    # Ahora extrae los elementos visibles en el carrusel
    items_carrusel = driver.find_elements(By.CLASS_NAME, clase_main_bestsellers)

    for item in items_carrusel:
        # Aquí puedes procesar cada item como desees, por ejemplo:
        print(item.text)

finally:
    # Cierra el navegador
    driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException

# Configuración del navegador
chrome_options = Options()
chrome_options.add_argument("--headless")  # Ejecuta el navegador en modo headless (sin interfaz gráfica)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Ruta al chromedriver
chromedriver_path = r'C:\Users\pabma\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe'

# Inicializa el navegador
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    # Espera a que la página cargue completamente
    driver.get('https://www.amazon.es/gp/bestsellers/?ref_=nav_cs_bestsellers')

    try:
        consent_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "sp-cc-accept"))
        )
        consent_button.click()
    except (TimeoutException, NoSuchElementException):
        print("No se encontró el botón de consentimiento de cookies.")

    # Localiza todas las filas del carrusel
    filas_carrusel = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-carousel-row-inner'))
    )
    print(filas_carrusel)

    # Para cada fila en el carrusel
    for fila in filas_carrusel:
        # Localiza todas las flechas dentro de esta fila
        try:
            flechas = fila.find_elements(By.CLASS_NAME, 'a-carousel-goto-nextpage')
            print(flechas)
            # Haz clic en cada flecha de la fila y extrae los elementos visibles
            for flecha in flechas:
                flecha.click()
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, clase_main_bestsellers))
                )

                # Extrae los elementos visibles en el carrusel por su nombre de clase
                items_carrusel = fila.find_elements(By.CLASS_NAME, clase_historial_bestsellers)

                for item in items_carrusel:
                    # Aquí puedes procesar cada item como desees, por ejemplo:
                    print(item.text)

        except NoSuchElementException as e:
            print(f"Error al localizar las flechas: {e}")

except TimeoutException as e:
    print(f"Tiempo de espera agotado para cargar los elementos: {e}")

finally:
    # Cierra el navegador
    driver.quit()


In [ ]:
items_carrusel